# Analisis Dominosmania

## Iniciamos la conexion a snowflake

In [1]:
from snowflake.snowpark import Session
from configparser import ConfigParser

config = ConfigParser()
config.read('config.ini')

connection_parameters: dict[str, int | str] = {
    'user' : config.get('SNOWFLAKE', 'USER'),
    'password' : config.get('SNOWFLAKE', 'PASSWORD'),
    'account' : config.get('SNOWFLAKE', 'ACCOUNT'),
    'database' : config.get('SNOWFLAKE', 'DATABASE'),
    'warehouse' : config.get('SNOWFLAKE', 'WAREHOUSE'),
    'schema' : config.get('SNOWFLAKE', 'SCHEMA'),
    'role' : config.get('SNOWFLAKE', 'ROLE'),
}

session = Session.builder.configs(connection_parameters).create()

In [2]:
import pandas as pd

clientes_adquiridos_cupon_adquisicion = pd.read_csv('./clientes_adquiridos_con_cupon_adquisicion.csv')

clientes_adquiridos_cupon_adquisicion = (
    session
    .create_dataframe(clientes_adquiridos_cupon_adquisicion)
    .with_column_renamed('FECHA', 'FECHA_ADQUISICION')
)

clientes_adquiridos_cupon_adquisicion.show()

---------------------------------------------------
|"EMAIL"                    |"FECHA_ADQUISICION"  |
---------------------------------------------------
|ivanero310.iggc@gmail.com  |2023-09-17           |
|arcelegion7@gmail.com      |2023-09-17           |
|olgamty1@gmail.com         |2023-09-17           |
|sbortiz.a@gmail.com        |2023-09-15           |
|reyfranc0o0o@gmail.com     |2023-09-12           |
|tatay_2188@hotmail.com     |2023-09-12           |
|dabadautrera83@gmail.com   |2023-09-12           |
|lic.cerv.dan18@gmail.com   |2023-09-13           |
|fc9907650@gmail.com        |2023-09-13           |
|dian109810@hotmail.com     |2023-09-13           |
---------------------------------------------------



In [4]:
from DP_transacciones_total import get_transacciones_dp_total
import snowflake.snowpark.functions as fn

transacciones_dp = get_transacciones_dp_total(session)

transacciones_despues_de_adquisicion = (
    clientes_adquiridos_cupon_adquisicion
    .join(transacciones_dp, on = 'EMAIL')
    .filter(fn.col('FECHA') > fn.col('FECHA_ADQUISICION'))
    .filter(fn.col('FECHA') <= fn.date_add(fn.col('FECHA_ADQUISICION'), 42))
)

volumen_total = clientes_adquiridos_cupon_adquisicion.agg(fn.count_distinct('EMAIL').alias('CLIENTES'))
volumen_con_recompra = transacciones_despues_de_adquisicion.agg(fn.count_distinct('EMAIL').alias('CLIENTES_CON_RECOMPRA'), fn.count_distinct('ORDER_ID').alias('TRANSACCIONES'))

volumen_recompra_y_frecuencia = (
    volumen_total
    .join(volumen_con_recompra)
    .with_column('Porcentaje con recompra', fn.col('CLIENTES_CON_RECOMPRA') / fn.col('CLIENTES'))
    .with_column('Frecuencia', fn.col('TRANSACCIONES') / fn.col('CLIENTES_CON_RECOMPRA'))
)

# volumen_recompra_y_frecuencia.to_pandas().to_csv('./analisis_dominosmania/volumen_recompra_y_frecuencia.csv')
volumen_recompra_y_frecuencia.show()

volumen_con_recompra_solo_con_cupon = (
    transacciones_despues_de_adquisicion
    .filter(~fn.col('TIENE_CUPON'))
    .agg(fn.count_distinct('EMAIL').alias('CLIENTES_CON_RECOMPRA'), fn.count_distinct('ORDER_ID').alias('TRANSACCIONES'))
)

volumen_con_recompra_solo_con_cupon.show()

-----------------------------------------------------------------------------------------------------
|"CLIENTES"  |"CLIENTES_CON_RECOMPRA"  |"TRANSACCIONES"  |"Porcentaje con recompra"  |"FRECUENCIA"  |
-----------------------------------------------------------------------------------------------------
|41893       |6259                     |16754            |0.149404                   |2.676785      |
-----------------------------------------------------------------------------------------------------

---------------------------------------------
|"CLIENTES_CON_RECOMPRA"  |"TRANSACCIONES"  |
---------------------------------------------
|3381                     |11966            |
---------------------------------------------



In [31]:
pd_dias_para_primera_recompra = (
    transacciones_despues_de_adquisicion
    .group_by(['FECHA_ADQUISICION', 'EMAIL'])
    .agg(fn.min(fn.col('FECHA')).alias('FECHA_PRIMERA_RECOMPRA'))
    .with_column('DIAS_PARA_PRIMERA_RECOMPRA', fn.daydiff(fn.col('FECHA_ADQUISICION'), fn.col('FECHA_PRIMERA_RECOMPRA')))
    .agg(fn.avg(fn.col('DIAS_PARA_PRIMERA_RECOMPRA')))
).to_pandas()

In [32]:
pd_dias_para_primera_recompra

,AVG(DIAS_PARA_PRIMERA_RECOMPRA)
0,-16.274295


## Fechas y cupones de las Dominosmanias

In [4]:
import pandas as pd

dominosmanias = session.create_dataframe(pd.read_csv('./dominosmanias.csv'))

dominosmanias_orden = session.create_dataframe(pd.read_csv('./dominosmanias_orden.csv'))

dominosmanias.show()
dominosmanias_orden.show()

---------------------------------------------------
|"NOMBRE"                  |"CUPON"  |"FECHA"     |
---------------------------------------------------
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-07  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-08  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-09  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-10  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-11  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-12  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-13  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-14  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-15  |
|pre_DOMINOSMANIA_Febrero  |NULL     |2023-02-16  |
---------------------------------------------------

---------------------------------------------
|"NOMBRE"                      |"POSICION"  |
---------------------------------------------
|pre_DOMINOSMANIA_Febrero      |1           |
|DOMINOSMANIA_Febrero          |2           |
|post_DOMINOSMANIA_Febrero     |3        

## Analisis de recompra

### Clientes que entraron por Dominosmania (Nuevos clientes y recurrentes)

In [7]:
from DP_transacciones_total import get_transacciones_dp_total
from general_primeras_compras import get_primeras_compras
from snowflake.snowpark.functions import lit, count_distinct, col
import snowflake.snowpark.functions as fn

transacciones_dp = get_transacciones_dp_total(session)

clientes_nuevos = (
    get_primeras_compras(transacciones_dp)
    .join(dominosmanias, on = 'FECHA')
    .select(['EMAIL', 'NOMBRE'])
    .distinct()
    .with_column('TIPO', lit('NUEVO'))
)

transacciones_previas = (
    dominosmanias
    .group_by('NOMBRE')
    .agg(fn.min('FECHA').alias('FECHA_FIN'))
    .join(transacciones_dp)
    .filter(col('FECHA') < col('FECHA_FIN'))
)

inactivos_al_inicio_del_periodo = (
    transacciones_previas
    .group_by('NOMBRE', 'EMAIL', 'FECHA_FIN')
    .agg(fn.max('FECHA').alias('FECHA_ULTIMA_COMPRA'))
    .with_column('FECHA_INICIO', fn.date_add(col('FECHA_FIN'), -180))
    .filter(col('FECHA_ULTIMA_COMPRA') <= col('FECHA_INICIO'))
)

recuperados = (
    transacciones_dp
    .join(dominosmanias, on = 'FECHA')
    .join(inactivos_al_inicio_del_periodo, on = ['EMAIL', 'NOMBRE'])
    .select(['EMAIL', 'NOMBRE'])
    .distinct()
    .with_column('TIPO', lit('RECUPERADO'))
)

clientes_existentes = (
    transacciones_dp
    .join(dominosmanias, on = 'FECHA')
    .join(clientes_nuevos, on = ['EMAIL', 'NOMBRE'], how = 'LEFTANTI')
    .join(recuperados, on = ['EMAIL', 'NOMBRE'], how = 'LEFTANTI')
    .select(['EMAIL', 'NOMBRE'])
    .distinct()
    .with_column('TIPO', lit('EXISTENTE'))
)

clientes_que_entraron_por_dominosmania = clientes_nuevos.union_all(clientes_existentes).union_all(recuperados)

clientes_que_entraron_por_dominosmania_volumen = (
    clientes_que_entraron_por_dominosmania
    .join(dominosmanias_orden, on = 'NOMBRE')
    .group_by(['NOMBRE', 'TIPO', 'POSICION'])
    .agg(count_distinct(col('EMAIL')).alias('CLIENTES'))
    .order_by(['POSICION', 'TIPO'])
)

clientes_que_entraron_por_dominosmania_volumen.to_pandas().to_csv('./analisis_dominosmania/clientes_que_entraron.csv')

clientes_que_entraron_por_dominosmania_volumen.show()

--------------------------------------------------------------------
|"NOMBRE"                   |"TIPO"      |"POSICION"  |"CLIENTES"  |
--------------------------------------------------------------------
|pre_DOMINOSMANIA_Febrero   |EXISTENTE   |1           |170547      |
|pre_DOMINOSMANIA_Febrero   |NUEVO       |1           |29819       |
|pre_DOMINOSMANIA_Febrero   |RECUPERADO  |1           |29083       |
|DOMINOSMANIA_Febrero       |EXISTENTE   |2           |196172      |
|DOMINOSMANIA_Febrero       |NUEVO       |2           |33446       |
|DOMINOSMANIA_Febrero       |RECUPERADO  |2           |34070       |
|post_DOMINOSMANIA_Febrero  |EXISTENTE   |3           |157934      |
|post_DOMINOSMANIA_Febrero  |NUEVO       |3           |25082       |
|post_DOMINOSMANIA_Febrero  |RECUPERADO  |3           |24871       |
|pre_DOMINOSMANIA_Junio     |EXISTENTE   |4           |156999      |
--------------------------------------------------------------------



In [14]:
from DP_transacciones_total import get_transacciones_dp_total
from general_primeras_compras import get_primeras_compras
from snowflake.snowpark.functions import lit, count_distinct, col
import snowflake.snowpark.functions as fn

transacciones_dp = get_transacciones_dp_total(session)

transacciones_durante_dominosmania = (
    transacciones_dp
    .join(dominosmanias, on = 'FECHA')
)

transacciones_durante_dominosmania_volumen = (
    transacciones_durante_dominosmania
    .join(dominosmanias_orden, on = 'NOMBRE')
    .group_by(['NOMBRE',
            #    'TIPO',
               'POSICION'])
    .agg(count_distinct(col('ORDER_ID')).alias('TRANSACCIONES'))
    .order_by(['POSICION',
            #    'TIPO'
               ])
)

transacciones_durante_dominosmania_venta = (
    transacciones_durante_dominosmania
    .join(dominosmanias_orden, on = 'NOMBRE')
    .group_by(['NOMBRE',
            #    'TIPO',
               'POSICION'])
    .agg(fn.sum('VENTA').alias('VENTA'))
    .order_by(['POSICION',
            #    'TIPO'
               ])
)

transacciones_durante_dominosmania_volumen.show()
transacciones_durante_dominosmania_venta.show()

---------------------------------------------------------------
|"NOMBRE"                      |"POSICION"  |"TRANSACCIONES"  |
---------------------------------------------------------------
|pre_DOMINOSMANIA_Febrero      |1           |250631           |
|DOMINOSMANIA_Febrero          |2           |293490           |
|post_DOMINOSMANIA_Febrero     |3           |226724           |
|pre_DOMINOSMANIA_Junio        |4           |229689           |
|DOMINOSMANIA_Junio            |5           |361392           |
|post_DOMINOSMANIA_Junio       |6           |224752           |
|pre_DOMINOSMANIA_Septiembre   |7           |214156           |
|DOMINOSMANIA_Septiembre       |8           |432966           |
|post_DOMINOSMANIA_Septiembre  |9           |212668           |
---------------------------------------------------------------

------------------------------------------------------------------
|"NOMBRE"                      |"POSICION"  |"VENTA"             |
---------------------------------

In [15]:
from DP_transacciones_total import get_transacciones_dp_total
from general_get_tiempo import get_tiempo
from general_primeras_compras import get_primeras_compras
from snowflake.snowpark.functions import lit, count_distinct, col
import snowflake.snowpark.functions as fn

transacciones_dp = get_transacciones_dp_total(session)

tiempo = get_tiempo(session)

transacciones_semanales = (
    transacciones_dp
    .join(tiempo, on = 'FECHA')
    .group_by('ANIO_ALSEA', 'SEM_ALSEA')
    .agg(fn.count_distinct('ORDER_ID').alias('TRANSACCIONES'))
)

transacciones_semanales_pd = transacciones_semanales.to_pandas()

transacciones_semanales_pd.to_csv('./analisis_dominosmania/transacciones_semanales.csv')

### Cuantos volvieron a comprar? Cual es su frecuencia?

In [69]:
from snowflake.snowpark.functions import max, col, count_distinct, date_add

transacciones_despues_de_dominosmania = (
    dominosmanias
    .group_by('NOMBRE')
    .agg(max('FECHA').alias('FECHA_FIN'))
    .join(clientes_que_entraron_por_dominosmania, on = 'NOMBRE')
    .join(transacciones_dp, on = 'EMAIL')
    .filter(col('FECHA') > col('FECHA_FIN'))
    .filter(col('FECHA') <= date_add(col('FECHA_FIN'), 42))
)

volumen_total = clientes_que_entraron_por_dominosmania.group_by(['NOMBRE', 'TIPO']).agg(count_distinct('EMAIL').alias('CLIENTES'))
volumen_con_recompra = transacciones_despues_de_dominosmania.group_by(['NOMBRE', 'TIPO']).agg(count_distinct('EMAIL').alias('CLIENTES_CON_RECOMPRA'), count_distinct('ORDER_ID').alias('TRANSACCIONES'))

volumen_recompra_y_frecuencia = (
    volumen_total
    .join(volumen_con_recompra, on = ['NOMBRE', 'TIPO'])
    .with_column('Porcentaje con recompra', col('CLIENTES_CON_RECOMPRA') / col('CLIENTES'))
    .with_column('Frecuencia', col('TRANSACCIONES') / col('CLIENTES_CON_RECOMPRA'))
    .join(dominosmanias_orden, on = 'NOMBRE')
    .order_by(['POSICION', 'TIPO'])
)

volumen_recompra_y_frecuencia.to_pandas().to_csv('./analisis_dominosmania/volumen_recompra_y_frecuencia.csv')
volumen_recompra_y_frecuencia.show()

PermissionError: [Errno 13] Permission denied: './analisis_dominosmania/volumen_recompra_y_frecuencia.csv'

In [6]:
from snowflake.snowpark.functions import max, date_add

transacciones_despues_de_dominosmania_con_cupon = (
    dominosmanias
    .group_by('NOMBRE')
    .agg(max('FECHA').alias('FECHA_FIN'))
    .join(clientes_que_entraron_por_dominosmania, on = 'NOMBRE')
    .join(transacciones_dp, on = 'EMAIL')
    .filter(col('FECHA') > col('FECHA_FIN'))
    .filter(col('FECHA') <= date_add(col('FECHA_FIN'), 42))
    .filter(~col('TIENE_CUPON'))
)

volumen_total = clientes_que_entraron_por_dominosmania.group_by(['NOMBRE', 'TIPO']).agg(count_distinct('EMAIL').alias('CLIENTES'))
volumen_con_recompra_con_cupon = transacciones_despues_de_dominosmania_con_cupon.group_by(['NOMBRE', 'TIPO']).agg(count_distinct('EMAIL').alias('CLIENTES_CON_RECOMPRA'), count_distinct('ORDER_ID').alias('TRANSACCIONES'))

volumen_recompra_y_frecuencia_con_cupon = (
    volumen_total
    .join(volumen_con_recompra_con_cupon, on = ['NOMBRE', 'TIPO'])
    .with_column('Porcentaje con recompra', col('CLIENTES_CON_RECOMPRA') / col('CLIENTES'))
    .with_column('Frecuencia', col('TRANSACCIONES') / col('CLIENTES_CON_RECOMPRA'))
    .join(dominosmanias_orden, on = 'NOMBRE')
    .order_by(['POSICION', 'TIPO'])
)

volumen_recompra_y_frecuencia_con_cupon.to_pandas().to_csv('./analisis_dominosmania/volumen_recompra_y_frecuencia.csv')
volumen_recompra_y_frecuencia_con_cupon.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------
|"NOMBRE"                   |"TIPO"     |"CLIENTES"  |"CLIENTES_CON_RECOMPRA"  |"TRANSACCIONES"  |"Porcentaje con recompra"  |"FRECUENCIA"  |"POSICION"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------
|pre_DOMINOSMANIA_Febrero   |EXISTENTE  |199630      |105014                   |222475           |0.526043                   |2.118527      |1           |
|pre_DOMINOSMANIA_Febrero   |NUEVO      |29819       |6180                     |10153            |0.207250                   |1.642880      |1           |
|DOMINOSMANIA_Febrero       |EXISTENTE  |230242      |120748                   |250912           |0.524440                   |2.077981      |2           |
|DOMINOSMANIA_Febrero       |NUEVO      |33446       |6759            

In [72]:
from snowflake.snowpark.functions import min, daydiff, avg

pd_dias_para_primera_recompra = (
    transacciones_despues_de_dominosmania
    .group_by(['NOMBRE', 'FECHA_FIN', 'EMAIL'])
    .agg(min(col('FECHA')).alias('FECHA_PRIMERA_RECOMPRA'))
    .with_column('DIAS_PARA_PRIMERA_RECOMPRA', daydiff(col('FECHA_FIN'), col('FECHA_PRIMERA_RECOMPRA')))
    .group_by('NOMBRE')
    .agg(avg(col('DIAS_PARA_PRIMERA_RECOMPRA')))
).to_pandas()

### Entendimiento de la composicion de ordenes de la promocionalidad (Dominosmania, otros cupones, full price)

In [31]:
from snowflake.snowpark.functions import concat, to_date, when, sum
from DP_transacciones_olo import get_transacciones_olo

descuentos = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACDESCUENTOS')
    .filter(
        col('FIIDDESCUENTOSRC').like(r'2D50%') |
        col('FIIDDESCUENTOSRC').like(r'D229%') |
        col('FIIDDESCUENTOSRC').like(r'DS199%')
    )
    .with_column_renamed('FIIDDESCUENTO', 'FCIDDESCUENTO')
    .with_column('DOMINOSMANIA', lit(True))
)

descuentos_ordenes = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFDESCUENTOSORDENES3')
    .join(descuentos, on = 'FCIDDESCUENTO', how='LEFT')
)

ordenes = (
    session
    .table('SEGMENT_EVENTS.ALOPERACIONES.TAFORDENES')
    .filter(col('FIIDCLIENTE').is_not_null())
    .filter(col('FIIDCLIENTE') != 0)
    .filter(col('FIIDMARCA').isin([4, 193]))
    .with_column('FECHA', to_date('FDIDDIA'))
    .with_column('ID_CLIENTE', concat(col('FCIDTIENDA'), col('FIIDCLIENTE')))
    .join(descuentos_ordenes, on = ['FCIDTIENDA', 'FITICKET', 'FDIDDIA'], how = 'LEFT')
    .with_column('PROMOCIONALIDAD', 
        when(
            col('FCIDDESCUENTO').is_null(), lit('FULL_PRICE')
        ).when(
            ~col('DOMINOSMANIA'), lit('OTROS_CUPONES')
        ).otherwise(
            lit('DOMINOSMANIA')
        )
    )
)

clientes = (
    session
    .table('SEGMENT_EVENTS.ALMODCOMUN.TACCLIENTES')
    .with_column('ID_CLIENTE', concat(col('FIIDTIENDA'), col('FIIDCLIENTE')))
    .join(ordenes, on = 'ID_CLIENTE')
)

transacciones_olo = get_transacciones_olo(session)

transacciones_olo_con_promocionalidad = (
    transacciones_olo
    .join(clientes, on = ['ID_CLIENTE', 'FECHA'])
    .join(dominosmanias, on = 'FECHA')
    .group_by(['PROMOCIONALIDAD', 'NOMBRE'])
    .agg(sum(col('VENTA')).alias('VENTA'), count_distinct(col('EMAIL')).alias('CLIENTES'), count_distinct('ORDER_ID').alias('TRANSACCIONES'))
)

venta_con_promocionalidad = transacciones_olo_con_promocionalidad.select(['PROMOCIONALIDAD', 'NOMBRE', 'VENTA']).to_pandas()
clientes_con_promocionalidad = transacciones_olo_con_promocionalidad.select(['PROMOCIONALIDAD', 'NOMBRE', 'CLIENTES']).to_pandas()
transacciones_con_promocionalidad = transacciones_olo_con_promocionalidad.select(['PROMOCIONALIDAD', 'NOMBRE', 'TRANSACCIONES'])

In [33]:
venta_con_promocionalidad.pivot(index = 'NOMBRE', columns = 'PROMOCIONALIDAD', values = 'VENTA')

PROMOCIONALIDAD,DOMINOSMANIA,FULL_PRICE
NOMBRE,,
DOMINOSMANIA_Febrero,9.903912e+07,7.064780e+06
DOMINOSMANIA_Junio,1.029894e+08,6.302909e+06
DOMINOSMANIA_Septiembre,9.594284e+07,2.847457e+06
post_DOMINOSMANIA_Febrero,7.041854e+07,5.363459e+06
post_DOMINOSMANIA_Junio,6.679909e+07,3.394663e+06
post_DOMINOSMANIA_Septiembre,4.358550e+07,2.024766e+06
pre_DOMINOSMANIA_Febrero,8.427915e+07,6.341799e+06
pre_DOMINOSMANIA_Junio,6.474895e+07,5.192399e+06
pre_DOMINOSMANIA_Septiembre,5.769475e+07,2.913636e+06


In [51]:
from snowflake.snowpark import DataFrame
from snowflake.snowpark.functions import substring, lower

def get_transacciones_cloud(session: Session) -> DataFrame:
    transacciones_cloud = (
        session.table('SEGMENT_EVENTS.DOMINOS_GOLO.VENTA_CLOUD')
        .filter(~col('STOREID').like('9%'))
        .filter(col('SOURCEORGANIZATIONURI').is_not_null())
        .filter(col('SOURCEORGANIZATIONURI').isin(['order.dominos.com', 'resp-order.dominos.com', 'iphone.dominos.mx', 'android.dominos.mx']))
        .with_column('FECHA', substring(col('STOREORDERID'), 1, 10))
        .with_column('EMAIL', lower(col('EMAIL')))
        .with_column('ORDER_ID', concat(col('FECHA'), col('STOREID'), col('STOREORDERID')))
        .with_column('VENTA', col('PAYMENTSAMOUNT') / 1.16)
        .with_column_renamed('STOREID', 'STORE_ID')
        # .select(['EMAIL', 'FECHA', 'VENTA', 'ORDER_ID', 'PHONE', 'STORE_ID'])
    )

    return transacciones_cloud

transacciones_cloud = get_transacciones_cloud(session)

composicion_transacciones = (
    transacciones_cloud
    .with_column(
        'PROMOCIONALIDAD',
        when(
            col('COUPONSCODE').like(r'%2D50%') |
            col('COUPONSCODE').like(r'%D229%') |
            col('COUPONSCODE').like(r'%DS199%')
        , lit('DOMINOSMANIA'))
        .when(
            col('COUPONSCODE').is_not_null(),
            lit('OTROS_CUPONES')
        ).otherwise('FULL_PRICE')
    )
    .join(dominosmanias, on = 'FECHA')
    .group_by(['PROMOCIONALIDAD', 'NOMBRE'])
    .agg(sum(col('VENTA')).alias('VENTA'), count_distinct(col('EMAIL')).alias('CLIENTES'), count_distinct('ORDER_ID').alias('TRANSACCIONES'))
    .join(dominosmanias_orden, on = 'NOMBRE')
    .order_by('POSICION', 'PROMOCIONALIDAD')
)

composicion_transacciones.to_pandas().to_csv('./analisis_dominosmania/composicion_transacciones.csv')

### Cuantos clientes se mantienen o se pierden?

In [46]:
from DP_transacciones_total import get_transacciones_dp_total
from general_rangos_de_fecha import get_rangos
from snowflake.snowpark import Session, DataFrame
from snowflake.snowpark.functions import col, count_distinct
from typing import Literal

def get_activos_en_serie_de_tiempo(session: Session, transacciones_total:DataFrame, serie_de_tiempo:list[Literal['ANIO_ALSEA', 'MES_ALSEA', 'SEM_ALSEA', 'FECHA']], intervalo_de_dias:int = 180) -> DataFrame:
    if ('EMAIL' not in transacciones_total.columns) and ('FECHA' not in transacciones_total.columns):
        raise KeyError('El DataFrame de transacciones debe contener las columnas "EMAIL" y "FECHA"')
    elif 'EMAIL' not in transacciones_total.columns:
        raise KeyError('El DataFrame de transacciones debe contener una columna "EMAIL"')
    elif 'FECHA' not in transacciones_total.columns:
        raise KeyError('El DataFrame de transacciones debe contener una columna "FECHA"')

    rangos = get_rangos(session, serie_de_tiempo, {'FECHA_FIN': 0, 'FECHA_INICIO': -intervalo_de_dias})

    res = (
        transacciones_total
        .join(rangos)
        .filter(col('FECHA_INICIO') <= col('FECHA'))
        .filter(col('FECHA') <= col('FECHA_FIN'))
        # .group_by('ANIO_ALSEA', 'SEM_ALSEA')
        # .agg(count_distinct('EMAIL').alias('ACTIVOS'))
    )

    return res

transacciones_dp_total = get_transacciones_dp_total(session)

activos_en_serie_de_tiempo = get_activos_en_serie_de_tiempo(session, transacciones_dp_total, ['ANIO_ALSEA', 'SEM_ALSEA']).filter(col('ANIO_ALSEA') == 2023)

se_mantienen_o_pierden = (
    clientes_que_entraron_por_dominosmania
    .filter(col('NOMBRE').like(r'%Febrero%'))
    .join(activos_en_serie_de_tiempo, on = 'EMAIL')
    .group_by('NOMBRE', 'ANIO_ALSEA', 'SEM_ALSEA')
    .agg(count_distinct(col('EMAIL')))
)

se_mantienen_o_pierden_pd = se_mantienen_o_pierden.to_pandas()

se_mantienen_o_pierden.show()

------------------------------------------------------------------------------------
|"NOMBRE"                   |"ANIO_ALSEA"  |"SEM_ALSEA"  |"COUNT(DISTINCT EMAIL)"  |
------------------------------------------------------------------------------------
|post_DOMINOSMANIA_Febrero  |2023          |31           |207887                   |
|DOMINOSMANIA_Febrero       |2023          |32           |263688                   |
|post_DOMINOSMANIA_Febrero  |2023          |46           |136983                   |
|pre_DOMINOSMANIA_Febrero   |2023          |39           |161317                   |
|pre_DOMINOSMANIA_Febrero   |2023          |35           |168558                   |
|post_DOMINOSMANIA_Febrero  |2023          |26           |207887                   |
|pre_DOMINOSMANIA_Febrero   |2023          |26           |229449                   |
|DOMINOSMANIA_Febrero       |2023          |30           |263688                   |
|DOMINOSMANIA_Febrero       |2023          |24           |263688 

In [50]:
se_mantienen_o_pierden_pd.pivot(index = ['NOMBRE'], columns=['SEM_ALSEA'], values = 'COUNT(DISTINCT EMAIL)').to_csv('./analisis_dominosmania/se_mantienen_o_pierden.csv')

In [63]:
from snowflake.snowpark.functions import upper, to_char

mx_power_sales_log = (
    session
    .table('SEGMENT_EVENTS.DOMINOS_OLO.MXPOWERSALESLOG')
    .with_column_renamed(col('ORDERNUMBER'), 'ORDER_NUMBER')
    .with_column_renamed(col('STORENUMBER'), 'LOCATION_CODE')
    .with_column_renamed(col('ORDERDATE'), 'FECHA')
    .with_column('EMAIL', lower(col('EMAIL')))
)

dpm_sales_full = (
    session
    .table('SEGMENT_EVENTS.DOMINOS_OLO.DPMSALES_FULL')
    .with_column_renamed(col('ORDER_DATE'), 'FECHA')
    .with_column('SOURCE_CODE', upper(col('SOURCE_CODE')))
    .filter(col('ORDER_STATUS_CODE') == 4)
    .filter(~col('LOCATION_CODE').isin(['13001', '13006', '13021', '11000']))
    .filter(col('SOURCE_CODE').isin(['ANDROID' , 'DESKTOP', 'IOS', 'MOBILE', 'WEB', 'ANDROID2', 'DESKTOP2', 'IOSAPP', 'MOBILE2', 'WHATSAPP']))
)

transacciones_olo = (
    mx_power_sales_log
    .join(dpm_sales_full, on = ['ORDER_NUMBER', 'LOCATION_CODE','FECHA'])
    .with_column('VENTA', col('ORDERFINALPRICE') / 1.16)
    .with_column('ORDER_ID', concat(to_char(col('FECHA')), col('LOCATION_CODE'), col('ORDER_NUMBER')))
    .with_column('ID_CLIENTE', concat(col('LOCATION_CODE'), col('CUSTOMER_CODE')))
    .with_column_renamed('PHONENUMBER', 'PHONE')
    .with_column_renamed('LOCATION_CODE', 'STORE_ID')
)

transacciones_olo

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"STOREPLACEORDERTIME"  |"PLACEORDERTIME"     |"STORE_ID"  |"STOREORDERID"      |"FUTUREORDERTIME"  |"CUSTOMERID"                              |"PHONE"  |"FIRSTNAME"  |"LASTNAME"         |"SERVICEMETHOD"  |"SOURCEORGANIZATIONURI"  |"COUPONSCODE"     |"PAYMENTSAMOUNT"  |"PAYMENTSTYPE"        |"PAYMENTSCARDTYPE"  |"PAYMENTSTRANSACTIONID"                             |"PAYMENTSPROVIDERID"  |"FILE_NAME"                        |"FECHA"     |"EMAIL"         

In [77]:
def get_transacciones_olo(session: Session) -> DataFrame:
    mx_power_sales_log = (
        session
        .table('SEGMENT_EVENTS.DOMINOS_OLO.MXPOWERSALESLOG')
        .with_column_renamed(col('ORDERNUMBER'), 'ORDER_NUMBER')
        .with_column_renamed(col('STORENUMBER'), 'LOCATION_CODE')
        .with_column_renamed(col('ORDERDATE'), 'FECHA')
        .with_column('EMAIL', lower(col('EMAIL')))
    )

    dpm_sales_full = (
        session
        .table('SEGMENT_EVENTS.DOMINOS_OLO.DPMSALES_FULL')
        .with_column_renamed(col('ORDER_DATE'), 'FECHA')
        .with_column('SOURCE_CODE', upper(col('SOURCE_CODE')))
        .filter(col('ORDER_STATUS_CODE') == 4)
        .filter(~col('LOCATION_CODE').isin(['13001', '13006', '13021', '11000']))
        .filter(col('SOURCE_CODE').isin(['ANDROID' , 'DESKTOP', 'IOS', 'MOBILE', 'WEB', 'ANDROID2', 'DESKTOP2', 'IOSAPP', 'MOBILE2', 'WHATSAPP']))
    )

    transacciones_olo = (
        mx_power_sales_log
        .join(dpm_sales_full, on = ['ORDER_NUMBER', 'LOCATION_CODE','FECHA'])
        .with_column('VENTA', col('ORDERFINALPRICE') / 1.16)
        .with_column('ORDER_ID', concat(to_char(col('FECHA')), col('LOCATION_CODE'), col('ORDER_NUMBER')))
        .with_column('ID_CLIENTE', concat(col('LOCATION_CODE'), col('CUSTOMER_CODE')))
        .with_column_renamed('PHONENUMBER', 'PHONE')
        .with_column_renamed('LOCATION_CODE', 'STORE_ID')
        .with_column('TIENE_CUPON', col('ORDERCOUPONSNUMBER') > 0)
    )

    return transacciones_olo

get_transacciones_olo(session).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------